# 第二次课后练习

<span style="color:black; font-weight:bold;">请将作业文件命名为 第二次课后练习+姓名+学号.ipynb, 例如 第二次课后练习+张三+1000000000.ipynb</span>

<span style="color:black; font-weight:bold;">在作业过程中觉得有心得或者自己拓展学习到有价值内容的，可以在文件名最后加一个#号。例如第二次课后练习+张三+1000000000+#.ipynb</span>

本次作业提交截止为周四晚。建议能完成的尽量提前提交以方便作业讲评。

## 零、 阅读代码预测-观察执行结果

In [5]:
n = 2
def func():
    print(n)

func()

def func2():
    print(n)
    n+=1
func2()

2


UnboundLocalError: local variable 'n' referenced before assignment

在func2中，由于没有用global声明n，所以默认是在函数中定义的新的局部变量，但在func2函数内n没有初始化赋值，所以会报错误

（改成n=1就不会报错，会认为是局部变量的赋值）

In [ ]:
#函数变量的作用域
def scope_test():
    def test_local():
        spam="Local" 
    def test_nonlocal():
        nonlocal spam
        spam="Nonlocal"
    def test_global():
        global spam
        spam="Global"
    spam="Test"
    test_local()
    print("1"+spam)
    test_global()
    print("2"+spam)
    test_nonlocal()
    print("3"+spam)


try:
    print(spam)
except:
    print("not defined")
scope_test()
print("4"+spam)

not defined
1Test
2Test
3Nonlocal
4Global


spam初始时在全局未定义，所以not defined（没有调用scope_test）
调用scope_test后，在这个闭包中spam定义为"Test"（局部变量）:

1.执行test_local，test_local中的赋值是在这个最内层函数的局部变量，外层的spam变量不变

2.执行test_global，test_global中的赋值由于全局没有spam变量，故这里初始化了spam变量并且修改为"Global"。但是返回外层函数后，输出行的spam默认从最近的开始逐渐向外找，故输出的仍是这个闭包内的spam而不是全局的spam。

3.执行test_nonlocal，test_nonlocal中的赋值是修改外层函数的变量，故外层的spam改变

4.回到全局的spam，在2.中初始化并修改了spam，根据此输出

In [2]:
def is_palme(n):
    n=str(n)
    m=n[::-1]
    return n==m

L = [12321,12343,213,242,433]

list(filter(lambda s: is_palme(s), L))

[12321, 242]

字符串反转等于自身 <==> 回文数

In [ ]:
def filter_data(data, threshold):
    for item in data:
        if data.index(item) > threshold:
            yield item

data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

filtered_data5 = filter_data(data, 5)

lst = [x for x in  filtered_data5]
lst

[7, 8, 9, 10]

把所有“这个数的数值第一次出现位置的下标>=6”的数取出来

In [ ]:
def fla(lst):
    for item in lst:
        if isinstance(item, list):  # 如果当前元素仍然是列表，递归调用fla
            yield from fla(item)
        else:
            yield item

nes_list = [[[1, 2], [3, 4]], 5, [6, 7, [8, 9]]]

for i in fla(nes_list):
    print(i)

1
2
3
4
5
6
7
8
9


有递归的生成器函数，递归地（即如果有列表嵌套，则递归进入直到为非列表对象）按原列表顺序生成列表拆散后的所有数

reduce的作用是将一个序列的值合并成一个值，如`reduce(lambda x, y: x+y, [1, 2, 3, 4, 5])`等价于`((((1+2)+3)+4)+5)`


In [12]:
from functools import reduce

text = """You are right, yes you are right"""

words = map(lambda s: s.lower(), filter(lambda s: s.isalpha(), text.split()))

fre = reduce(lambda d, word: {**d, word: d.get(word, 0) + 1}, words, {})

print(fre)

{'you': 2, 'are': 2, 'yes': 1, 'right': 1}


s.isalpha():判断s是否是英文字母（故排除right，），故第三行后words是迭代器：（you are yes you are right）

第四行按顺序做，**d表示这个字典，后面是d中按顺序到目前为止出现word的次数（未出现则为默认值0），相当于是增加键值对或更新键值对（因为每个键对应一个值），故为统计单词在这个迭代器中出现次数

第四行最后的{}表示字典初始值

In [ ]:
# 实现函数组合器 # 考试不会考函数组合器

def compose(*funcs):
    return reduce(lambda f,g: lambda x: f(g(x)), funcs)

add1 = lambda x: x+1
mul2 = lambda x: x*2

compose(mul2, add1)(3)

8

*func传可变数量的对象；reduce按顺序做，mul2(add1(3))=8

In [ ]:
funcs = [lambda x: x+i for i in range(3)]
print([f(10) for f in funcs])

[12, 12, 12]


#### <mark> 列表推导式中的 lambda 函数捕获了循环变量i的引用（而不是值），但由于 Python 的后期绑定行为，所有生成的 lambda 函数在被调用时都会使用循环结束后的 i 值。<mark>

#### <mark> Python 的后期绑定指函数或闭包在其被调用时才确定它引用的变量值，而不是在定义时就确定。这种行为特别容易在使用循环和闭包时出现，因为闭包会捕获变量的引用，而不是变量的当前值。 <mark>

故最终i=2，所有lambda x 引用同一个i，当函数被调用时循环已经结束。故func = [lambda x: x + 2 for i in range(3)]

f遍历funcs，计算函数值10 + 2，故为10+2，10+2，10+2组成的列表

------

修改方式：将变量用默认参数给函数。
```python
funcs = [lambda x, i=i: x + i for i in range(3)]
print([f(10) for f in funcs])
```

In [15]:
def counter_factory(initial=0):
    count = initial  # 闭包变量存储状态
    def increment(step=1):
        nonlocal count
        count += step
        return count
    def reset():
        nonlocal count
        count = 0
        return count
    def get():
        return count
    return {'increment': increment, 'reset': reset, 'get': get}

counter1 = counter_factory(10)
counter2 = counter_factory()

print("Counter1:", counter1['increment'](5), counter1['get']())  
print("Counter2:", counter2['increment'](), counter2['get']())   
counter1['reset']()
print("After reset:", counter1['get'](), counter2['get']())      

Counter1: 15 15
Counter2: 1 1
After reset: 0 1


传入参数都是不可变对象，不会出现交叉。

counter1中，初始count=10，先调用字典中‘increment’对应的函数，外层局部变量count会修改（增加传入参数5），变成15，然后再get是修改后的count=15，然后reset同样nonlocal后会修改count变成0，最后get就得到0

counter2中，初始count=默认参数0，然后increment会让count增加默认参数1，之后仿上count变成1并输出

## 一、 函数基础练习-补充代码完善流程

### 1. 二分查找 代码填空

设计一个函数，实现二分查找 输入是一个升序的整数列表 和 一个数，返回这个数列表中的下标，没有查到打印 NA 

In [6]:
# 返回 x 在 arr 中的索引，如果不存在返回 -1
def binarySearch (arr, l, r, x): 
  
    # 基本判断
    if r >= l: 
  
        mid = (l + r) >> 1                 # 填空：找到中点下标
  
        # 元素整好的中间位置
        if arr[mid] == x: 
            return mid 
          
        # 元素小于中间位置的元素，只需要再比较左边的元素
        elif arr[mid] > x: 
            return binarySearch(arr, l, mid, x)              #  填空：递归向左半边 
  
        # 元素大于中间位置的元素，只需要再比较右边的元素
        else: 
            return binarySearch(arr, mid + 1, r, x)             #填空：递归向右半边 
  
    else: 
        # 不存在
        print('NA')
        return -1

# 测试数组
arr = [ 2, 3, 4, 10, 40 ] 
x = 10
  
# 函数调用
result = binarySearch(arr, 0, len(arr)-1, x) 
assert result == 3

### 2. 实现数字列表的归并排序 补充代码块

例如：
输入：[12,3,4,6,7,1,8,5]
输出：[1,3,4,5,6,7,8,12]

In [7]:
def mergesort(seq):
    """归并排序"""
    if len(seq) <= 1:
        return seq
    mid = len(seq) // 2  # 将列表分成更小的两个列表
    # 分别对左右两个列表进行处理，分别返回两个排序好的列表
    left = mergesort(seq[:mid])
    right = mergesort(seq[mid:])
    # 对排序好的两个列表合并，产生一个新的排序好的列表
    return merge(left, right)

def merge(left, right):
    """合并两个已排序好的列表，产生一个新的已排序好的列表"""
    result = []  # 新的已排序好的列表
    i = 0  # 下标
    j = 0
    
    # 对两个列表中的元素 两两对比。
    # 将最小的元素，放到result中，并对当前列表下标加1
    while i < len(left) and j < len(right):
     # 补充该循环体代码，实现功能要求 todo：   
        if left[i] < right[j]:
            result += [left[i]]
            i += 1
        else:
            result += [right[j]]
            j += 1

    result += left[i:]  # <-list的加法即合并
    result += right[j:] 
    return result

seq = [12,3,4,6,7,1,8,5]
result = mergesort(seq)
assert result ==  [1, 3, 4, 5, 6, 7, 8, 12]

### 3.生成器实现两个序列的交错输出 补充实现函数体
    编写一个生成器函数，接受两个序列作为参数，并生成一个新序列，该序列交错包含原来两个序列的元素。例如，给定序列`[1, 3, 5]`和`[2, 4, 6]`，生成的序列应该是`[1, 2, 3, 4, 5, 6]`。（最好不要使用`zip`函数）

In [8]:
def interleave(seq1, seq2):
    pass             # todo
    i = 0
    while i < len(seq1) and i < len(seq2):
        yield seq1[i]
        yield seq2[i]
        i += 1


assert list(interleave([1, 2, 3], [4, 5, 6,7 ])) == [1, 4, 2, 5, 3, 6]

### 4.使用装饰器计算函数运行的时间并输出，格式如下；

    如调用func(1,2,3)时，输出func(1,2,3):[0.06199502944946289]s;

    如调用func(1,[1,2],{1:[1,2,3]})时，输出func(1,[1, 2],{1: [1, 2, 3]}):[0.06698012351989746]s

提示：参数的字符化只考虑简单数据类型和包含简单数据类型的容器，并且只考虑参数*args，可以用map(str,args)转换为字符串。有基础的可以考虑试下**kwargs。

打印函数的名字可以使用func.__name__，获取当前时间可以使用time包

In [9]:
import time
def log_time(func):
    def inner(*args):
        pass
        start = time.time()
        func(*args)
        end = time.time()
        print(f"{func.__name__}{args}:[{end - start}]s")
    return inner

@log_time
def time_cosume(*args):
    return sum(range(1,1000000))

time_cosume(1,2,3,4,[1,2,3,4])
# time_cosume(1,2,3,4,[1,2,3,4],{1:[1,2]})

time_cosume(1, 2, 3, 4, [1, 2, 3, 4]):[0.014480829238891602]s
